# 03 - DuckDB Queries
DuckDB is an in-process SQL OLAP database management system. [duckdb.org](http://duckdb.org)

With the `timeseries`, `geomerty`, `crosswalk` and `attribute` table data stored in Parquet files, using DuckDB, we can explore the data using SQL.

NOTE: The example code provided is not safe from SQL injection.  It is provided to demonstrate the power of DuckDB to query data stored in Parquet files. Also, this is not a web service, so their is no risk in this case.

In [1]:
# Import required libraries
import duckdb
from pathlib import Path

In [2]:
# Again. set some files paths
CACHE_DIR = Path(Path.home(), "shared", "teehr-workshop")
STUDY_DIR = Path(CACHE_DIR, "post-event-example")
USGS = Path(STUDY_DIR, "timeseries/usgs/*.parquet")
MEDIUM_RANGE_MEM1 = Path(STUDY_DIR, "timeseries/medium_range_mem1/*.parquet")
SHORT_RANGE = Path(STUDY_DIR, "timeseries/short_range/*.parquet")
CROSSWALK = Path(STUDY_DIR, "geo/usgs_nwm22_crosswalk.parquet")
GEOMETRY = Path(STUDY_DIR, "geo/usgs_geometry.parquet")

To demonstrate how DuckDB can be used to query the Parquet fioles, we will write and execute a few simple SQL queries. First, lets just see how many rows are in one of the timeseries "tables".

In [3]:
duckdb.query(f"""
    SELECT count(*) FROM read_parquet('{MEDIUM_RANGE_MEM1}')
;""").to_df()

,count_star()
0,230227200


Now lets see how many unique `referece_times` are in the timeseries table.

In [4]:
duckdb.query(f"""
    SELECT count(DISTINCT reference_time) 
    FROM read_parquet('{MEDIUM_RANGE_MEM1}')
;""").to_df()

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

,count(DISTINCT reference_time)
0,120


How about getting those `reference_times` as a dataframe

In [5]:
duckdb.query(f"""
    SELECT DISTINCT reference_time 
    FROM read_parquet('{MEDIUM_RANGE_MEM1}')
;""").to_df()

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

,reference_time
0,2022-12-29 06:00:00
1,2022-12-24 00:00:00
2,2022-12-29 12:00:00
3,2023-01-08 00:00:00
4,2022-12-18 18:00:00
...,...
115,2022-12-20 18:00:00
116,2023-01-02 06:00:00
117,2023-01-06 18:00:00
118,2023-01-04 12:00:00


Now, lets see a single row from from the timeseries data.

In [6]:
%%time
duckdb.query(f"""
    SELECT * FROM read_parquet('{SHORT_RANGE}') LIMIT 1;
""").to_df()

CPU times: user 28.9 ms, sys: 38.1 ms, total: 67 ms
Wall time: 146 ms


,value_time,location_id,value,measurement_unit,reference_time,configuration,variable_name,__index_level_0__
0,2023-01-02 17:00:00,nwm22-7086109,0.22,m3/s,2023-01-02 16:00:00,short_range,streamflow,0


In [7]:
%%time
duckdb.query(f"""
    SELECT * FROM 
        read_parquet('{SHORT_RANGE}')
    WHERE 
        location_id = 'nwm22-7086109'
        AND value_time = '2023-01-02 17:00:00'
        AND reference_time = '2023-01-02 16:00:00'
;""").to_df()

CPU times: user 140 ms, sys: 138 ms, total: 278 ms
Wall time: 543 ms


,value_time,location_id,value,measurement_unit,reference_time,configuration,variable_name,__index_level_0__
0,2023-01-02 17:00:00,nwm22-7086109,0.22,m3/s,2023-01-02 16:00:00,short_range,streamflow,0


Now, lets do something a little more complex just to show the power of SQL.  Lets join the observed USGS data to the short range forecast data and select a single forecast.

In [9]:
%%time
duckdb.query(f"""
    SELECT 
        u.location_id as primary_location_id,
        sr.location_id as secondary_location_id,
        sr.reference_time as reference_time,
        u.value_time as value_time,
        u.value as primary_value,
        sr.value as secondary_value,
    FROM read_parquet('{SHORT_RANGE}') sr
    JOIN read_parquet('{CROSSWALK}') cw 
        ON sr.location_id = cw.secondary_location_id 
    JOIN read_parquet('{USGS}') u 
        ON cw.primary_location_id = u.location_id
        AND sr.value_time = u.value_time
    WHERE
        primary_location_id = 'usgs-10336676' AND
        sr.reference_time = '2023-01-02 16:00:00' AND
        primary_value > 0
    ORDER BY 
        sr.value_time DESC
    LIMIT 10;
""").to_df()

CPU times: user 602 ms, sys: 131 ms, total: 733 ms
Wall time: 642 ms


,primary_location_id,secondary_location_id,reference_time,value_time,primary_value,secondary_value
0,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 10:00:00,1.087367,0.04
1,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 09:00:00,1.121347,0.04
2,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 08:00:00,1.121347,0.05
3,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 07:00:00,1.155327,0.06
4,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 06:00:00,1.155327,0.08
5,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 05:00:00,1.189308,0.10
6,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 04:00:00,1.189308,0.13
7,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 03:00:00,1.189308,0.17
8,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 02:00:00,1.226119,0.22
9,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 01:00:00,1.226119,0.29


This starts to get trciky as the queries get more complex.  This is where TEEHR queries library comes in.
![teehr.png](../images/teehr.png)